In [ ]:
import os
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import time
from concurrent.futures import ProcessPoolExecutor

In [ ]:
# Crear el directorio si no existe
output_dir = '../../data/processed/Batch5/CaseB'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Definir los parámetros generales del modelo
params = {
    'pv_price': 80,                # Precio del PV en EUR/kW
    'bess_price': 200,             # Precio del BESS en EUR/kWh
    'pv_opex': 3,                  # Costos operativos del PV en EUR/kW
    'bess_opex': 6,                # Costos operativos del BESS en EUR/kWh
    'pv_co2': 33,                  # Emisiones de CO2 del PV en kgCO2eq/kW
    'bess_co2': 100,               # Emisiones de CO2 del BESS en kgCO2eq/kWh
    'pv_opex_co2': 0,              # Emisiones operativas de CO2 del PV
    'bess_opex_co2': 0,            # Emisiones operativas de CO2 del BESS
    'discount_rate': 0.0485,       # Tasa de descuento
    'lifetime_project': 32,        # Vida útil del proyecto en años
    'lifetime_bess': 8,            # Vida útil del BESS en años
    'degradation_rate': 0.025      # Tasa de degradación
}

In [ ]:
# Cargar datos desde archivos CSV
price = pd.read_csv('../../data/raw/PriceCurve_SE3_2021.csv', sep=';')
co2_pro = pd.read_csv('../../data/raw/production_emissions.csv')
co2_con = pd.read_csv('../../data/raw/consumption_emissions.csv')
pv = pd.read_csv('../../data/raw/pv_sam.csv')
load = pd.read_csv('../../data/raw/LoadCurve.csv', sep=';')


In [ ]:
# Ajustar y preparar los datos
data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv
data['Load'] = data['Load'] * 1000  # Convertir de MW a kW
data['Price'] = data['Price'] / 1000  # Ajustar unidades de precio
data['CO_2_eq'] = data['CO_2_eq'] / 1000  # Ajustar unidades de CO2
data['Hour'] = data['Hour'].astype('int')


In [ ]:
# Duplicar las últimas filas para completar el año (corrección de tamaño)
duplicated_rows = pd.concat([data.iloc[[-1]]] * 49, ignore_index=True)
data = pd.concat([data, duplicated_rows], ignore_index=True)


In [ ]:
# Definir los conjuntos y límites del modelo
flows_global = [
    'P_PV_to_Load', 'P_PV_to_BESS', 'P_PV_curtailment', 'P_PV_to_Grid',
    'P_BESS_to_Load', 'P_BESS_to_Grid', 'P_Grid_to_Load', 'P_Grid_to_BESS'
]
hours_global = list(range(8760 + 24))
time_window = 48  # Ventana de tiempo para optimización iterativa
num_iterations = 8760 / time_window
grid_production = 1000000  # Capacidad máxima de producción de la red (kW)

In [ ]:
# Función para verificar si un archivo existe (evita recalcular si ya está hecho)
def file_exists(filename):
    return os.path.isfile(filename)

In [ ]:
# Función para resolver el modelo de optimización
def solve_optimization(bess_cap, bess_multiple, solar_multiple, curve):
    if not file_exists(f'{output_dir}/{curve}_{int(solar_multiple)}_{bess_multiple}_{bess_cap}.csv'):
        results = pd.DataFrame(index=hours_global, columns=flows_global + ['SoC'])
        p_solar = (solar_multiple * 3000) / 100000
        p_bess = bess_multiple * 3000
        t_bess = bess_cap
        if t_bess != 0:
            p_bess = bess_multiple * 3000
        else:
            p_bess = 0
        e_bess = p_bess * t_bess
        soc_initial = e_bess * 0.5
        start_time = time.time()
        print(f'PV Multiple: {solar_multiple}x Peak Load [MW] || BESS Multiples: {bess_multiple}x Peak Load [MW]; {bess_cap} hr (at rated power) == {bess_multiple * 3 * bess_cap} [MWh]')
        if curve == 'price':
            print(f'Optimizado para: Price-curve')
        elif curve == 'co2':
            print(f'Optimizado para: CO2eq-curve')

        for iteration in range(1, int(num_iterations) + 1):
            hours = list(range((iteration - 1) * time_window, iteration * time_window))
            demand = {hour: data['Load'][hour] for hour in hours}
            pv_production = {hour: data['solar_PV'][hour] * p_solar for hour in hours}
            costs_keys = [(flow, hour) for flow in flows_global for hour in hours]
            costs_economic = {}
            costs_environmental = {}

            for key in costs_keys:
                flow, hour = key
                if flow in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
                    costs_economic[key] = data['Price'][hour]
                    costs_environmental[key] = data['CO_2_eq'][hour]
                elif flow == 'P_PV_to_Grid':
                    costs_economic[key] = -data['Price'][hour] / 1000
                    costs_environmental[key] = -data['CO_2_eq'][hour] / 1000
                elif flow in ['P_PV_to_Load', 'P_PV_to_BESS', 'P_BESS_to_Load']:
                    costs_economic[key] = 0
                    costs_environmental[key] = 0
                elif flow == 'P_BESS_to_Grid':
                    costs_economic[key] = -data['Price'][hour] / 1000
                    costs_environmental[key] = -data['CO_2_eq'][hour] / 1000
                elif flow == 'P_PV_curtailment':
                    costs_economic[key] = 1
                    costs_environmental[key] = 1

            model = pe.ConcreteModel()
            model.flows = pe.Set(initialize=flows_global, ordered=True)
            model.hours = pe.Set(initialize=hours, ordered=True)
            model.grid_production = pe.Param(initialize=grid_production)
            model.demand = pe.Param(model.hours, initialize=demand)
            model.pv_production = pe.Param(model.hours, initialize=pv_production)
            model.Efficiency_charge = pe.Param(model.hours, initialize=0.98)
            model.Efficiency_inverter = pe.Param(model.hours, initialize=0.97)
            model.Efficiency_discharge = pe.Param(model.hours, initialize=0.96)
            model.SoCmin = pe.Param(initialize=e_bess * 0.1)
            model.SoCmax = pe.Param(initialize=e_bess * 0.9)
            model.MaxCharge = pe.Param(initialize=p_bess)
            model.SoCinitial = pe.Param(initialize=soc_initial if iteration == 1 else float(results['SoC'].iloc[-1]) if not pd.isna(results['SoC'].iloc[-1]) else e_bess * 0.1)
            model.arbitrageLimit = pe.Param(model.hours, initialize=float(data['Load'].max()) * 1.2)
            model.costs = pe.Param(model.flows, model.hours, initialize=costs_economic if curve == 'price' else costs_environmental, default=1)
            model.p = pe.Var(model.flows, model.hours, domain=pe.NonNegativeReals)
            model.SoC = pe.Var(model.hours, domain=pe.Reals, bounds=(model.SoCmin, model.SoCmax))
            model.Sw_charge = pe.Var(model.hours, domain=pe.Binary)
            model.Sw_discharge = pe.Var(model.hours, domain=pe.Binary)
            model.objective = pe.Objective(sense=pe.minimize, expr=sum(model.p[f, t] * model.costs[f, t] for f in model.flows for t in model.hours))

            loadFullfilment = {t: (model.p['P_PV_to_Load', t] * 0.97 + model.p['P_BESS_to_Load', t] * 0.96 * 0.97 + model.p['P_Grid_to_Load', t]) == model.demand[t] for t in model.hours}
            pvProduction = {t: (model.p['P_PV_to_Load', t] + model.p['P_PV_to_BESS', t] + model.p['P_PV_to_Grid', t] + model.p['P_PV_curtailment', t]) == model.pv_production[t] for t in model.hours}
            arbitrageFlow = {t: (model.p['P_BESS_to_Grid', t] * 0.96 * 0.97 + model.p['P_PV_to_Grid', t] * 0.97) <= model.arbitrageLimit[t] for t in model.hours}
            gridFlow = {t: (model.p['P_Grid_to_Load', t] + model.p['P_Grid_to_BESS', t]) <= model.grid_production for t in model.hours}
            bessChargeFlow = {t: (model.p['P_Grid_to_BESS', t] + model.p['P_PV_to_BESS', t]) <= model.Sw_charge[t] * model.MaxCharge for t in model.hours}
            bessDischargeFlow = {t: (model.p['P_BESS_to_Grid', t] + model.p['P_BESS_to_Load', t]) <= model.Sw_discharge[t] * model.MaxCharge for t in model.hours}
            limitValidation = {t: model.Sw_charge[t] + model.Sw_discharge[t] <= 1 for t in model.hours}

            def storage_state(model, t):
                if t == model.hours.first():
                    return model.SoC[t] == model.SoCinitial + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * 0.98 * 0.97) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t]) / (0.96 * 0.97))
                else:
                    return model.SoC[t] == model.SoC[t-1] + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * 0.98 * 0.97) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t]) / (0.96 * 0.97))

            model.demandRule = pe.Constraint(model.hours, expr=loadFullfilment)
            model.pvProductionRule = pe.Constraint(model.hours, expr=pvProduction)
            model.arbitrageRule = pe.Constraint(model.hours, expr=arbitrageFlow)
            model.gridRule = pe.Constraint(model.hours, expr=gridFlow)
            model.chargeRule = pe.Constraint(model.hours, expr=bessChargeFlow)
            model.dischargeRule = pe.Constraint(model.hours, expr=bessDischargeFlow)
            model.limitValidation = pe.Constraint(model.hours, expr=limitValidation)
            model.charge_state = pe.Constraint(model.hours, expr=storage_state)

            solver = po.SolverFactory('glpk')
            solver.options['tmlim'] = 15
            solver.options['mipgap'] = 0.05
            modelresults = solver.solve(model, tee=False)

            for flow in flows_global + ['SoC']:
                for hour in hours:
                    if flow == 'SoC':
                        results.at[hour, 'SoC'] = model.SoC[hour].value
                    else:
                        results.at[hour, flow] = model.p[flow, hour].value
                results[flow] = results[flow].astype(float)

            results['sum_power_flows'] = results.P_PV_to_Load + results.P_BESS_to_Load + results.P_Grid_to_Load
            results['sum_power_flows'] = results['sum_power_flows'].astype(float)

            for param in model.component_data_objects(pe.Param, active=True):
                model.del_component(param)
            for variable in model.component_data_objects(pe.Var, active=True):
                model.del_component(variable)
            for cons in model.component_data_objects(pe.Constraint, active=True):
                model.del_component(cons)
            for obj in model.component_data_objects(pe.Objective, active=True):
                model.del_component(obj)
            model.del_component(model.hours)
            model.del_component(model.flows)

        results = results.reset_index()
        results = results.rename(columns={'index': 'Hour'})
        results.to_csv(f'{output_dir}/{curve}_{int(solar_multiple)}_{bess_multiple}_{t_bess}.csv', sep=',', index=False, columns=None)
        end_time = time.time()
        execution_time = end_time - start_time
        hours, remainder = divmod(execution_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        print("Execution time: {:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds)))
        print('-' * 50)


In [ ]:
# Obtener el número de núcleos de la CPU
num_workers = os.cpu_count() - 2 # 2 hilos para el sistema operativo y otros procesos para evitar que se bloquee

# Paralelizar la ejecución de múltiples problemas de optimización
with ProcessPoolExecutor(max_workers=num_workers) as executor:
    futures = []
    for bess_cap in [0, 1, 2, 3, 4, 5, 6, 7, 8]:
        for bess_multiple in [1, 2, 3, 4, 5, 6, 7, 8, 10]:
            for solar_multiple in [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 15]:
                for curve in ['price', 'co2']:
                    futures.append(executor.submit(solve_optimization, bess_cap, bess_multiple, solar_multiple, curve))
    for future in futures:
        future.result()